# **Eclat for Movie Recomendation Analysis**

Menggunakan algoritma Eclat untuk mendapatkan rekomendasi film berdasarkan aktor, sutradara dan genre

Eclat pada dasarnya adalah versi apriori yang disederhanakan, yang hanya mempertimbangkan support, daripada menggunakan ketiganya seperti apriori, yaitu support, Confidence , dan Lift. Oleh karena itu, dapat diimplementasikan dengan cara yang sama seperti apriori, dengan hanya mempertimbangkan Support selama visualisasi

In [ ]:
pip install apyori

In [ ]:
import pandas as pd
import numpy as np
from apyori import apriori
import random

## Preprocessing the data 

In [ ]:
url = 'https://raw.githubusercontent.com/furqoncreative/data-mining-eclat/master/movie_metadata.csv'
dataset=pd.read_csv(url)
dataset.fillna(value=0,axis=1,inplace=True)
dataset.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,0,Doug Walker,0.0,0.0,131.0,0.0,Rob Walker,131.0,0.0,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens,8,143,0,0.0,0,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,0.0,0,0,0,0.0,0.0,12.0,7.1,0.00,0


In [ ]:
dataset.shape

(5043, 28)

Menampung dataset dalam bentuk array

In [ ]:
movieData=[]
for i in range(0,1000):
    movieData.append([str(dataset.values[i,j]) for j in range(0,28)])

In [ ]:
movieData[:1]

[['Color',
  'James Cameron',
  '723.0',
  '178.0',
  '0.0',
  '855.0',
  'Joel David Moore',
  '1000.0',
  '760505847.0',
  'Action|Adventure|Fantasy|Sci-Fi',
  'CCH Pounder',
  'Avatar\xa0',
  '886204',
  '4834',
  'Wes Studi',
  '0.0',
  'avatar|future|marine|native|paraplegic',
  'http://www.imdb.com/title/tt0499549/?ref_=fn_tt_tt_1',
  '3054.0',
  'English',
  'USA',
  'PG-13',
  '237000000.0',
  '2009.0',
  '936.0',
  '7.9',
  '1.78',
  '33000']]

Membuat sampel transaksi kebiasan user menonton film dalam 30 hari.
Dengan asumsi bahwa film yang ditonton selanjutnya:
1. memiliki aktor yang sama dari film sebelumnya, atau
2. memiliki sutradara yang sama dari film sebelumnya, atau
3. memiliki genre yang sama dari film sebelumnya

In [ ]:
def getNextMovie(s):
    for i in range(0,1000):
      if (movieData)[i][10] == s[10] or (movieData)[i][9] == s[1] or (movieData)[i][9] == s[9]:
        if movieData[i][11] != s[11]:
          return movieData[i]
        else:
          return random.choice(movieData)
      else: 
        return random.choice(movieData)

In [ ]:
movieTransactions=[]
userTransactions=[]
for i in range(0,1000):
    firtsMovie = (random.choice(movieData))
    userTransactions.append([])
    movieTransactions.append([])
    movieTransactions[i].append(getNextMovie(firtsMovie))
    for j in range(0,30):
      movieTransactions[i].append(getNextMovie(movieTransactions[i][j]))
      userTransactions[i].append(movieTransactions[i][j][11].replace(u'\xa0', u''))

In [ ]:
userTransactions[:3]

[['Alvin and the Chipmunks: Chipwrecked',
  'World War Z',
  'The Other Guys',
  'Avatar',
  'Hitch',
  'The Kid',
  'Assassins',
  'Christmas with the Kranks',
  'Cloud Atlas',
  'Harry Potter and the Deathly Hallows: Part I',
  'The Interpreter',
  'The 13th Warrior',
  'The Fast and the Furious',
  'Battleship',
  'The Perfect Storm',
  'The Chronicles of Narnia: Prince Caspian',
  'Transformers',
  'Deep Blue Sea',
  'Fantasia 2000',
  'The Departed',
  'Casper',
  'Night at the Museum',
  'The Negotiator',
  'Little Nicky',
  'The Matrix Reloaded',
  'Green Lantern',
  'Deep Impact',
  'Batman & Robin',
  'The Mummy: Tomb of the Dragon Emperor',
  'The Legend of Zorro'],
 ['Basic Instinct 2',
  'Leatherheads',
  'Creepshow',
  'Spanglish',
  'Cinderella Man',
  'AVP: Alien vs. Predator',
  'Green Lantern',
  'The Score',
  'Robin Hood',
  'Spectre',
  'Stardust',
  'Journey 2: The Mysterious Island',
  'Bee Movie',
  'Monsters vs. Aliens',
  'Eraser',
  'The Beach',
  'Bowfinger',

## Training the Eclat model

In [ ]:
# min_support = film ditonton 1x sehari * 30 hari / 1000 user = 0.003
# min_confidence: setidaknya 20%
# min_lift = 3

from apyori import apriori
rules=apriori(userTransactions,min_support=0.003,min_confidence=0.2,min_lift=3,min_length=3,max_length=3)

List hasil data training:

In [ ]:
results=list(rules)
results

[RelationRecord(items=frozenset({'47 Ronin', 'Batman & Robin'}), support=0.006, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Batman & Robin'}), items_add=frozenset({'47 Ronin'}), confidence=0.20689655172413793, lift=6.269592476489028)]),
 RelationRecord(items=frozenset({'47 Ronin', "The Emperor's New Groove"}), support=0.005, ordered_statistics=[OrderedStatistic(items_base=frozenset({"The Emperor's New Groove"}), items_add=frozenset({'47 Ronin'}), confidence=0.2173913043478261, lift=6.587615283267458)]),
 RelationRecord(items=frozenset({'A Beautiful Mind', 'Pacific Rim'}), support=0.005, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Pacific Rim'}), items_add=frozenset({'A Beautiful Mind'}), confidence=0.20833333333333334, lift=8.333333333333334)]),
 RelationRecord(items=frozenset({'Moneyball', 'A Civil Action'}), support=0.005, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Moneyball'}), items_add=frozenset({'A Civil Action'}), confidence=0.263

## Visualization of Results

In [ ]:
movie,r1,support=[],[],[]
for result in results:
    movie.append(tuple(result[2][0][0])[0])
    r1.append(tuple(result[2][0][1])[0])
    support.append(result[1])

In [ ]:
columns=list(zip(movie,r1,support))

Converting into dataframe

In [ ]:
result_df=pd.DataFrame(columns)
result_df.columns=['Film yang dipilih','Rekomendasi','Support']
result_df

,Film yang dipilih,Rekomendasi,Support
0,Batman & Robin,47 Ronin,0.006
1,The Emperor's New Groove,47 Ronin,0.005
2,Pacific Rim,A Beautiful Mind,0.005
3,Moneyball,A Civil Action,0.005
4,The Long Kiss Goodnight,Abraham Lincoln: Vampire Hunter,0.005
...,...,...,...
1269,The Day After Tomorrow,The Tale of Despereaux,0.003
1270,The Avengers,The Hobbit: The Desolation of Smaug,0.003
1271,The Devil's Advocate,Waterworld,0.003
1272,The Hobbit: The Desolation of Smaug,Wyatt Earp,0.003


Diurutkan berdasarkan "Support"

In [ ]:
result_df=result_df.sort_values('Support',ascending=False)
result_df

,Film yang dipilih,Rekomendasi,Support
17,End of Days,Avatar,0.035
23,The League of Extraordinary Gentlemen,Avatar,0.034
20,Star Wars: Episode II - Attack of the Clones,Avatar,0.034
21,Star Wars: Episode III - Revenge of the Sith,Avatar,0.027
18,Man of Steel,Avatar,0.026
...,...,...,...
571,Avatar,Town & Country,0.003
572,Four Christmases,The X Files,0.003
573,Funny People,Star Wars: Episode II - Attack of the Clones,0.003
574,Gangs of New York,Star Wars: Episode II - Attack of the Clones,0.003
